In [2]:
from my_functions import *

In [3]:
file= uproot.open("/home/ruben/Escritorio/BachelorThesisRuben/Data/Muon_trees.root") #opening the Root file with Uproot 

MuonTree_ZeroBias=file["MuonTree_ZeroBias;1"]

In [4]:
eta=MuonTree_ZeroBias.arrays(["LVL1Muon_eta"], entry_start=2000, entry_stop=3000)
phi=MuonTree_ZeroBias.arrays(["LVL1Muon_phi"], entry_start=2000, entry_stop=3000)

In [5]:
a=all_events_jTower_dr(eta,phi,MuonTree_ZeroBias,MuonTree_ZeroBias,"jTower_eta","jTower_phi")

a

<Array [[], [], [], [], ..., [], [], [], []] type='1000 * var * var * float64'>

In [6]:
isAll_below_threshold(a,0.4)

KeyboardInterrupt: 